## 6-12-6 sided Failure surface for six- and four-parameter Paul-Mohr-Coulomb 

In [1]:
import numpy as np
from load import load_data
from convert import convert

## For six-parameter Paul-Mohr-Coulomb criterion
from pmc import *
from brute_force import planes_def

## For four-parameter Paul-Mohr-Coulomb criterion
#from pmc_4p import *
#from brute_force_4p import planes_def_PMC

# Input

In [2]:
n= 13 # For Dunnville sandsone >> see load_data.py for other rocks
data = load_data(n)
d = convert(data)

## For six-parameter Paul-Mohr-Coulomb criterion
P1, P2, SP1, SP2 = planes_def(data,d)
print('Six parameter Paul-Mohr-Coulomb solution')
print('P1 plane : V0_P1 = {}, phi_c = {}, phi_e = {}, S = {}'.format(P1.Vo,P1.phyC*180/np.pi,P1.phyE*180/np.pi,SP1))
print('P2 plane : V0_P2 = {}, phi_c = {}, phi_e = {}, S = {}'.format(P2.Vo,P2.phyC*180/np.pi,P2.phyE*180/np.pi,SP2))
print('General S = {}'.format((SP1+SP2)/2))
print('\n')
## For four-parameter Paul-Mohr-Coulomb criterion
#fit_pts = np.concatenate([data[1:4,:],data[5:8,:]])
#P1, P2, SP1, SP2 = planes_def_PMC(data,fit_pts)
#print('Six parameter Paul-Mohr-Coulomb solution')
#print('P1 plane : V0_P1 = {}, phi_c = {}, phi_e = {}, S = {}'.format(P1.Vo,P1.phyC*180/np.pi,P1.phyE*180/np.pi,SP1))
#print('P2 plane : V0_P2 = {}, phi_c = {}, phi_e = {}, S = {}'.format(P2.Vo,P2.phyC*180/np.pi,P2.phyE*180/np.pi,SP2))
#print('General S = {}'.format((SP1+SP2)/2))
#print('\n')


/Users/heloisefuselier/Dropbox/PDM/PDM/python/pmc.py:152: RuntimeWarning: invalid value encountered in arcsin
  self.phyC = arcsin(3*bc/(6*self.Vo+bc))



Six parameter Paul-Mohr-Coulomb solution
P1 plane : V0_P1 = 86.88604900932947, phi_c = 14.386812418194536, phi_e = 12.113037667372204, S = 14.96247381857528
P2 plane : V0_P2 = 11.135694447843807, phi_c = 34.333113745238236, phi_e = 34.74696686352787, S = 6.118649968601993
General S = 10.540561893588636




# Paul-Mohr-Coulomb failure surface construction 

In [3]:
eq_pts_transisiton_P12 = np.zeros((6,3,3))
for i in range(6):
    eq_pts_transisiton_P12[i] = np.array([get_plane_normal_6_cycle(P1,i),get_plane_normal_6_cycle(P1,i-1),get_plane_normal_6_cycle(P2,i)])

ones = np.ones((6,3))
pts_transisiton_P12 = np.linalg.solve(eq_pts_transisiton_P12, ones).transpose()

eq_pts_P2 = np.array([get_plane_normal_6_cycle(P2,0),get_plane_normal_6_cycle(P2,1),get_plane_normal_6_cycle(P2,2)])
pts_P2 = np.linalg.solve(eq_pts_P2,np.ones((3,1)))

p_trans = (pts_transisiton_P12[0,:]+pts_transisiton_P12[1,:]+pts_transisiton_P12[2,:])/3

sig_pts = np.concatenate((P1.pts,P2.pts),axis=1) #miss pts_P2
pts_with_largest_coord = sig_pts[:,np.argmax(np.max(sig_pts, axis=0))]
dist = np.dot(np.ones(3)/np.sqrt(3),pts_with_largest_coord)*1.1

pts_P1_far = p_plane_intersection_6(P1,dist)

# Draw 3D failure surface

In [4]:
## Figure initialization
%matplotlib qt
import matplotlib.pyplot as plt
import matplotlib.font_manager
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
plt.rcParams.update({
    'pgf.rcfonts': False,
})
plt.rcParams['font.family'] = 'serif'
alf_inter = 1
alf_plane = 0.5

In [5]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(d['sig2'],d['sig3'],d['sig1'],'k.')
#ax.plot(*P1.pts,'k.')
#ax.plot(*P2.pts,'k.')
#ax.plot(d['sig2'],d['sig3'],d['sig1'],'k.')
ax.plot(*pts_P2,'ko',alpha=0)
ax.plot(*pts_transisiton_P12,'ro',alpha=alf_inter)
ax.plot(*pts_P1_far,'bo',alpha=alf_inter)
ax.set_xlabel('[MPa]')
ax.set_ylabel('[MPa]')
ax.set_zlabel('[MPa]')
def get_verts(coord):
    return [list(zip(*cord))]


## Draw P2 plane

for i in range(6):
    cord = np.zeros((3,3))
    cord[:,0] = pts_P2[:,0]
    cord[:,1] = pts_transisiton_P12[:,i]
    cord[:,2] = pts_transisiton_P12[:,(i+1)%6]
    
    verts = get_verts(cord)
    poly = Poly3DCollection(verts,linewidths=1, alpha=alf_plane)
    poly.set_facecolor('r')
    ax.add_collection3d(poly)
    
    
## Draw P1 plane

for i in range(6):
    cord = np.zeros((3,4))
    cord[:,0] = pts_transisiton_P12[:,i]
    cord[:,1] = pts_transisiton_P12[:,(i+1)%6]
    cord[:,2] = pts_P1_far[:,(i+1)%6]
    cord[:,3] = pts_P1_far[:,i%6]
    
    verts = get_verts(cord)
    poly = Poly3DCollection(verts,linewidths=1, alpha=alf_plane)
    poly.set_facecolor('b')
    ax.add_collection3d(poly)
    
    
ax.grid(b=None)
ax.set_zticks([0,40,80,120,160])
ax.set_xticks([0,40,80,120,160])
ax.set_yticks([0,40,80,120,160])
fig.show()

# Pi-plane for Paul-Mohr-Coulomb

In [6]:
# Base transformation
new_base = np.zeros((3,3))
new_z = np.array([1,1,1])
new_y = np.array([-1,-1,2])
new_x = np.array([-1,1,0])
new_base[:,0] = new_x /np.linalg.norm(new_x )
new_base[:,1] = new_y /np.linalg.norm(new_y )
new_base[:,2] = new_z /np.linalg.norm(new_z )
new_base_transform = np.linalg.inv(new_base)

# Failure surface planes in the new base
#new_sig = new_base_transform @ sig_pts
new_transi = new_base_transform @ pts_transisiton_P12
new_P2 = new_base_transform @ pts_P2
new_far_P1 = new_base_transform @ pts_P1_far

transi_dist_min = np.min(new_transi[2,:])
transi_dist_max = np.max(new_transi[2,:])
transi_dist_mid = (transi_dist_min+transi_dist_max)/2
near_inter = p_plane_intersection_6(P2,transi_dist_min*0.4)
near_transP2 = p_plane_intersection_6(P2,transi_dist_mid*1)
near_transP1 = p_plane_intersection_6(P1,transi_dist_mid*1)
new_near = new_base_transform @ near_inter
new_transP2 = new_base_transform @ near_transP2
new_transP1 = new_base_transform @ near_transP1

transi_dist_max = np.max(new_transi[2,:])
t = 0.5
dist = transi_dist_max*t + transi_dist_min*(1-t)
#offset = 2 if np.allclose(new_transi[0,0],transi_dist_max) else 0
offset = 2 if p_trans[0]>p_trans[1] else 0
middle_inter = p_plane_intersection_12(P1,P2,dist,offset)
new_middle = new_base_transform @ middle_inter

# Plot True-triaxial data and their fitted plane
o_sig = d['o'][:,:3].transpose()
o_pts = np.zeros(o_sig.shape)
o_pts[0,:] = o_sig[1,:]
o_pts[1,:] = o_sig[2,:]
o_pts[2,:] = o_sig[0,:]
new_o_pts = new_base_transform @ o_pts

p = np.unique(d['o'][:,3])
pts_tt1p = p_planes(p[0],P1,P2,p_trans,offset)
pts_tt2p = p_planes(p[1],P1,P2,p_trans,offset)
pts_tt3p = p_planes(p[3],P1,P2,p_trans,offset)
pts_tt1 = p_planes(p[2],P1,P2,p_trans,offset)
new_tt1p = new_base_transform @ pts_tt1p
new_tt2p = new_base_transform @ pts_tt2p
new_tt3p = new_base_transform @ pts_tt3p
new_tt1 = new_base_transform @ pts_tt1

In [7]:
## Plot of True-triaxial data and their fitted plane (Dunnville Sandstone)
plt.close()
#plt.axis('equal')
p1, = plt.plot(new_o_pts[0,:3],new_o_pts[1,:3],'r*', label='p={} MPa'.format(p[0]))
p2, = plt.plot(new_o_pts[0,3],new_o_pts[1,3],'b.',label='p={} MPa'.format(p[1]))
p3, = plt.plot(new_o_pts[0,4:6],new_o_pts[1,4:6],'gp',label='p={} MPa'.format(p[3]))
p4, = plt.plot(new_o_pts[0,6],new_o_pts[1,6],'yX',label='p={} MPa'.format(p[2]))

#p5, = plt.fill(new_middle[0,:],new_middle[1,:],edgecolor='k',fill=False,label='12 sided - P2 and P1')
plt.fill(new_tt1p[0,:],new_tt1p[1,:],edgecolor='r',fill=False)
plt.fill(new_tt2p[0,:],new_tt2p[1,:],edgecolor='b',fill=False)
plt.fill(new_tt3p[0,:],new_tt3p[1,:],edgecolor='g',fill=False)
plt.fill(new_tt1[0,:],new_tt1[1,:],edgecolor='y',fill=False)
plt.legend(handles=[p1, p2, p3, p4])
x = np.linspace(0,70)
zer = [0]*len(x)
v = np.array([x,x,zer])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k--')
v = np.array([x,zer,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k--')
v = np.array([zer,x,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k--')

x = np.linspace(-90,0)
zer = [0]*len(x)
v = np.array([x,x,zer])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k-')
v = np.array([x,zer,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k-')
v = np.array([zer,x,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k-')

plt.xlabel('[MPa]')
plt.ylabel('[MPa]')
plt.axis('equal')
#plt.axis('off')
plt.show()

In [8]:
## Plot 6 sided domain, 12 sided domain and 6 sided domain
plt.close()
p1, = plt.fill(new_far_P1[0,:],new_far_P1[1,:],edgecolor='b',fill=False,label='6 sided - P1')
p2, = plt.fill(new_near[0,:],new_near[1,:],edgecolor='r',fill=False,label='6 sided - P2')
plt.fill(new_transP2[0,:],new_transP2[1,:],edgecolor='r',fill=False)
plt.fill(new_transP1[0,:],new_transP1[1,:],edgecolor='b',fill=False)
p3, = plt.fill(new_middle[0,:],new_middle[1,:],edgecolor='k',fill=False,label='12 sided - P2 and P1')
plt.legend(handles=[p1, p2, p3])

x = np.linspace(0,100)
zer = [0]*len(x)
v = np.array([x,x,zer])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k--')
v = np.array([x,zer,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k--')
v = np.array([zer,x,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k--')

x = np.linspace(-110,0)
zer = [0]*len(x)
v = np.array([x,x,zer])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k-')
v = np.array([x,zer,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k-')
v = np.array([zer,x,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k-')

plt.xlabel('[MPa]')
plt.ylabel('[MPa]')
plt.axis('equal')
#plt.axis('off')
plt.show()

In [48]:
plt.close()